## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import random
# Import the kerastuner library
import keras_tuner as kt
import numpy as np

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
application_df['IS_SUCCESSFUL'].value_counts()

IS_SUCCESSFUL
1    18261
0    16038
Name: count, dtype: int64

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_type_count = application_df['APPLICATION_TYPE'].value_counts()
application_type_count

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_count = 500
application_types_to_replace = application_type_count.loc[application_type_count < cutoff_count].index.to_list()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_type_counts = application_df['CLASSIFICATION'].value_counts()
classification_type_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_type_counts.loc[classification_type_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_classification = 1000
classifications_to_replace = classification_type_counts.loc[
    classification_type_counts < cutoff_classification].index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [10]:
application_df['STATUS'].value_counts()

STATUS
1    34294
0        5
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_with_dummies = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 
                                        'AFFILIATION', 
                                        'CLASSIFICATION', 
                                        'USE_CASE', 
                                        'ORGANIZATION', 
                                        'INCOME_AMT', 
                                        'SPECIAL_CONSIDERATIONS',
                                        ], dtype=int)

In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df_with_dummies['IS_SUCCESSFUL']     # the binary target array

X = application_df_with_dummies.drop(columns='IS_SUCCESSFUL', axis=1)    # input features DataFrame

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [13]:
X_train.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
10679,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5052,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
9990,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
25173,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11405,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [14]:
X_train.head()

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
10679,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5052,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
9990,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
25173,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
11405,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [16]:
n_input_features = X_train_scaled.shape[1]  # the number of columns (features) in the X_train_scaled df
n_nodes_layer_1 = int(2 * n_input_features ) # scale the number of features in the first layer by number of features and a scalar
n_nodes_layer_2 = int(3 * n_input_features) # scale the number of features in the first layer by number of features and a scalar
activation_func_init = 'relu'



In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_init = tf.keras.models.Sequential(name='initial_model')

# First hidden layer
nn_init.add(tf.keras.layers.Dense(units = n_nodes_layer_1, 
                             input_dim = n_input_features,
                             activation = activation_func_init,
                             name = 'Layer_1'   
                            )
        )



# Second hidden layer
nn_init.add(tf.keras.layers.Dense(units = n_nodes_layer_2, 
                             activation = activation_func_init,
                             name='Layer_2'
                             )
        )

# Output layer
nn_init.add(tf.keras.layers.Dense(units = 1, 
                             activation='sigmoid',
                             name = 'Output',
                             )
        )

# Check the structure of the model
nn_init.summary()

Model: "initial_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Layer_1 (Dense)             (None, 86)                3784      
                                                                 
 Layer_2 (Dense)             (None, 129)               11223     
                                                                 
 Output (Dense)              (None, 1)                 130       
                                                                 
Total params: 15137 (59.13 KB)
Trainable params: 15137 (59.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
# compile the model
nn_init.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# create checkpoint callback for every 5 batches
init_model_checkpoint = tf.keras.callbacks.ModelCheckpoint("./checkpoints/init_model/",
                                                           save_weights_only = True,
                                                           save_freq = 5
                                                           )

# Train the model
fit_model = nn_init.fit(X_train_scaled, 
                        y_train, 
                        epochs=30,
                        callbacks = init_model_checkpoint)

Epoch 1/30
804/804 [==============================] - 3s 3ms/step - loss: 0.5707 - accuracy: 0.7213
Epoch 2/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5553 - accuracy: 0.7292
Epoch 3/30
804/804 [==============================] - 3s 4ms/step - loss: 0.5525 - accuracy: 0.7308
Epoch 4/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5500 - accuracy: 0.7316
Epoch 5/30
804/804 [==============================] - 3s 3ms/step - loss: 0.5500 - accuracy: 0.7322
Epoch 6/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5482 - accuracy: 0.7324
Epoch 7/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7328
Epoch 8/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5468 - accuracy: 0.7334
Epoch 9/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5466 - accuracy: 0.7346
Epoch 10/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5461 - accuracy: 0.7349

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_init.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5546 - accuracy: 0.7254 - 210ms/epoch - 784us/step
Loss: 0.5545585751533508, Accuracy: 0.7253644466400146


In [20]:
# save the model to HDF5 as per homework instructions
# this is considered a legacy option now as per warning message
nn_init.save("./models/AlphabetSoupCharity.h5")    

/Users/benwruck/miniconda3/envs/py310/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
# this is the recommended method for saving as per keras documentation which is a zipped archive which contains the h5 too!
# https://keras.io/guides/serialization_and_saving/
nn_init.save("./models/AlphabetSoupCharity.keras")